In [170]:
# let's use the brown corpus
from nltk.corpus import brown
data = brown.tagged_sents(tagset='universal')

In [171]:
from collections import defaultdict
count = defaultdict(int)
all_tags = set([])
for sentence in data[:20000]:
    prev_tag = '$'
    count[prev_tag] += 1
    for word,tag in sentence:
        count[(word.lower(), tag)] += 1
        count[(tag, prev_tag)] += 1
        count[tag] += 1
        prev_tag = tag
        all_tags.add(tag)

In [172]:
from __future__ import division
prob = defaultdict(lambda :0.000001)
for item in count:
    if type(item)==tuple:
        prob[item] = count[item]/count[item[1]]

In [173]:
all_tags = list(all_tags)

In [176]:
import numpy as np
from math import log
tot = 0
matched = 0
for sentence in data[20000:20500]:
    P = np.zeros((len(all_tags),len(sentence)))
    parent = np.zeros((len(all_tags),len(sentence)), dtype='int')
    pred = []
    for w in range(len(sentence)):
        for t in range(len(all_tags)):
            if w==0:
                P[t][w] = prob[(sentence[w][0].lower(),all_tags[t])]*prob[(all_tags[t],'$')] # 2nd formula
            else:
                mx_prob = -1.0*float('inf')
                for j in range(len(all_tags)):
                    if P[j][w-1]*prob[(all_tags[t],all_tags[j])] > mx_prob:
                        mx_prob = P[j][w-1]*prob[(all_tags[t],all_tags[j])]
                        best_tag = j
                P[t][w] = prob[(sentence[w][0].lower(),all_tags[t])]*mx_prob  # 1st formula
                parent[t,w] = best_tag
    currnt_tag = np.argmax(P[:,-1])
    pred.append(all_tags[currnt_tag])
    for k in range(len(sentence)-1,0,-1):
        pred.append(all_tags[parent[currnt_tag,k]])
        currnt_tag = parent[currnt_tag,k]
    pred = pred[::-1]
    for l in range(len(sentence)):
        if pred[l]==sentence[l][1]:
            matched += 1
        tot += 1

In [177]:
matched/tot

0.9386808367071525